### Retrieve Site and Parse HTML

In [ ]:
#%pip install requests
import requests
from bs4 import BeautifulSoup


url = "https://mykbostats.com/players/1"
apikey = "b327144420a12f944bbdd4863c9fc0bffb28a1e3" 
params = {
    "url": url,
    "apikey": apikey,
    "js_render": "true",
    "premium_proxy": "true",
}
response = requests.get("https://api.zenrows.com/v1/", params=params)

#print(response.content[:500])

soup = BeautifulSoup(response.content, 'html.parser')

### Player Name and Team

In [ ]:
t = soup.select('title')[0].text.strip()
player = t[:t.find(' - ')]

teams = {'Doosan': 'Doosan Bears', 
         'Hanwha': 'Hanwha Eagles', 
         'Kia': 'Kia Tigers', 
         'Kiwoom': 'Kiwoom Heroes', 
         'KT': 'KT Wiz', 
         'LG': 'LG Twins', 
         'Lotte': 'Lotte Giants', 
         'NC': 'NC Dinos', 
         'Samsung': 'Samsung Lions', 
         'SSG': 'SSG Landers'}

### Table Headers

In [ ]:
# extract table header information
header = soup.select('thead th')
for i in range(len(header)):
  header[i] = header[i].text.strip()

# remove 'Game Stats' data
header = header[:header.index("Date")]
header = ['Name'] + header

### Get Table Contents 
Write to contents to dataframe, then write to CSV. 

In [ ]:
#%pip install pandas
import pandas as pd
from unicodedata import numeric


#to handle unicode data in table
def uni_to_num (unicode):
    if ('(' in unicode):
       return unicode
    elif len(unicode) == 0:
        return None
    elif len(unicode) == 1:
        num = numeric(unicode)
    elif unicode[-1].isdigit():
        # normal number, ending in [0-9]
        num = float(unicode)
    else:
        # Assume the last character is a vulgar fraction
        num = float(unicode[:-1]) + numeric(unicode[-1])
    return num


#data to be inserted into dataframe later
temp = []

# Parse row data and add to temp
rows = soup.select('tbody tr')

for r in rows:
  if (r.select_one('.left').text.strip() == 'Career'):
    break

  t = [player, r.select_one('.left').text.strip(), r.select_one('nobr').text.strip()]  
  for i in r.select('td')[2:]:
    t.append(uni_to_num(i.text.strip()))
  temp.append(t)


# init dataframe
data = pd.DataFrame(data = temp, columns = header)

# write to CSV
data.to_csv('data.csv')

# read CSV
#df = pd.read_csv('.csv')

## Python Script

In [47]:
#%pip install ...
import pandas as pd
import requests
from bs4 import BeautifulSoup
from unicodedata import numeric
from time import sleep

def uni_to_num (unicode):
    '''
    Given string of unicode, convert into numerics.
    '''
    
    if ('(' in unicode):
       return unicode
    elif len(unicode) == 0:
        return None
    elif len(unicode) == 1:
        num = numeric(unicode)
    elif unicode[-1].isdigit():
        # normal number, ending in [0-9]
        num = float(unicode)
    else:
        # Assume the last character is a vulgar fraction
        num = float(unicode[:-1]) + numeric(unicode[-1])

    return num



def get_row_data (rows, player):
    '''
    Given HTML code of table with data and tuple containing player name and role, extract data and return as dataframe.
    '''
    
    rows = soup.select('tbody tr')

    temp = []
    for r in rows:
        # break loop to exclude data from Career onwards
        if (r.select_one('.left').text.strip() == 'Career'):
            break

        t = [player[0], r.select_one('.left').text.strip(), r.select_one('nobr').text.strip()]  
        for i in r.select('td')[2:]:
            t.append(uni_to_num(i.text.strip()))
        temp.append(t)

    return pd.DataFrame(data = temp, columns = header)



def get_header (soup):
    '''
    Given a BeautifulSoup object of HTML code, extract table heading information as list of strings
    '''
    
    header = soup.select('thead th')
    for i in range(len(header)):
        header[i] = header[i].text.strip()

    # remove 'Game Stats' data
    header = header[:header.index("Date")]
    
    return ['Name'] + header



def get_player (soup):
    '''
    Given BeautifulSoup object with HTML code, return tuple with player name and role (pitcher/batter) 
    '''
    
    t = soup.select('title')[0].text.strip()
    
    name = t[:t.find(' KBO')]
    
    if ("Pitching" in t):
        role = "Pitcher"
    elif ("Batting" in t):
        role = "Batter"
    
    return (name, role)



def get_website (url):
    '''
    Given string containing url of website, return BeautifulSoup object with parsed HTML code
    '''

    # generated from ZenRows.com
    apikey = "b327144420a12f944bbdd4863c9fc0bffb28a1e3" 
    params = {
        "url": url,
        "apikey": apikey,
        "js_render": "true",
        "premium_proxy": "true",
    }

    response = requests.get("https://api.zenrows.com/v1/", params=params)
    print(response.status_code)

    return BeautifulSoup(response.content, 'html.parser')



def fix_team_names (df):
    '''
    Given dataframe with player information, adjust Team names column to represent the full team name
    '''
    teams = {'Doosan': 'Doosan Bears', 
            'Hanwha': 'Hanwha Eagles', 
            'Kia': 'Kia Tigers', 
            'Kiwoom': 'Kiwoom Heroes', 
            'KT': 'KT Wiz', 
            'LG': 'LG Twins', 
            'Lotte': 'Lotte Giants', 
            'NC': 'NC Dinos', 
            'Samsung': 'Samsung Lions', 
            'SSG': 'SSG Landers'}
    
    for i in df.index:
        df.loc[i, 'Team'] = teams[df.loc[i, 'Team']]






for i in range(1, 5): 
    url = "https://mykbostats.com/players/" + str(i)
    soup = get_website(url)
    
    player = get_player(soup)
    header = get_header(soup)
    rows = soup.select('tbody tr')
    data = get_row_data(rows, player)

    fix_team_names(data)

    if (player[1] == "Pitcher"):
        data.to_csv('KBO_Pitchers.csv', mode='a')

    if (player[1] == "Batter"):
        data.to_csv('KBO_Batters.csv', mode='a')

    sleep(5) #website crawl-delay


    




402


IndexError: list index out of range